In [25]:
#I got the idea of how to read a .zst file from https://github.com/pushshift/zreader/tree/master and changed the fields based on our project's requirements.
import pandas as pd
import csv
import warnings
import openpyxl
warnings.filterwarnings("ignore")

In [26]:
import zstandard as zstd

class Zreader:

    def __init__(self, file, chunk_size=10000000):
        self.fh = open(file,'rb')
        self.chunk_size = chunk_size
        self.dctx = zstd.ZstdDecompressor(max_window_size=2147483648)
        self.reader = self.dctx.stream_reader(self.fh)
        self.buffer = ''


    def readlines(self):
        while True:
            chunk = self.reader.read(self.chunk_size).decode(encoding="latin-1")
            if not chunk:
                break
            lines = (self.buffer + chunk).split("\n")

            for line in lines[:-1]:
                yield line

            self.buffer = lines[-1]

In [ ]:
import ujson as json


# Adjust chunk_size as necessary -- defaults to 16,384 if not specified
reader = Zreader("RS_2022-04.zst", chunk_size=10000000)

list_lines=[]
j=0
# Read each line from the reader
for line in reader.readlines():
    obj = json.loads(line)
    sub_name=(obj['subreddit'].lower()).strip() 
    if ((sub_name=="books") or (sub_name=="suggestmeabook")):
        list_lines.append([obj['author'], obj['subreddit'], obj['created_utc'],obj['title'],obj['id'],
           obj['score'], obj['send_replies']])
    if j%10000000==0:
        print(j)
    j+=1

In [28]:
df = pd.DataFrame(list_lines, columns = ['author', 'subreddit','created_utc','title','id','score','send_replies'])

In [29]:
import xlsxwriter
df.to_excel("submissions_2022/april_2022.xlsx", engine='xlsxwriter')